<a href="https://colab.research.google.com/github/Alice049/formrecognizer/blob/main/form_recognizer_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install azure-ai-formrecognizer==3.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.4/228.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.4 MB/s eta 0:00:00


In [25]:
import os
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
endpoint="https://form001.cognitiveservices.azure.com/"
key="2dc786624694418c8e4219b88d5d0116"

def format_bounding_region(bounding_regions):
  if not bounding_regions:
    return "N/A"
  return ",".join("Page #{}:{}".format(region.page_number,format_polygon(region.polygon))for region in bounding_regions)

def format_polygon(polygon):
  if not polygon:
    return "N/A"
  return",".join(["[{},{}]".format(p.x,p.y) for p in polygon])

def analyze_general_documents():
  docUrl="https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-layout.pdf"
  document_analysis_client=DocumentAnalysisClient(endpoint=endpoint,credential=AzureKeyCredential(key))

  poller= document_analysis_client.begin_analyze_document_from_url(
      "prebuilt-document",docUrl)
  result=poller.result()

  for style in result.styles:
    if style.is_handwritten:
      print("Document contains handwritten content:")
      print(",".join([result.content[span.offset:span.offset+span.length] for span in style.spans]))
  print("----key-value pair found in document----")
  for kv_pair in result.key_value_pairs:
    if kv_pair.key:
      print(
          "key '{}' found within '{}' bounding regions".format(
              kv_pair.key.content,
              format_bounding_region(kv_pair.key.bounding_regions)
          )
      )

    if kv_pair.value:
      print(
          "value '{}' found within '{}' bounding regions \n".format(
              kv_pair.value.content,
              format_bounding_region(kv_pair.value.bounding_regions),
          )
      )
  for page in result.pages:
    print("----Analyzing document from page #{}----".format(page.page_number))
    print(
        "Page has width:{} and height: {},measured with unit:{}".format(
            page.width,page.height,page.unit
        )
    )
    for line_idx,line in enumerate(page.lines):
      print(
          "...Line #{} has text content '{}' within box '{}'".format(
              line_idx,
              line.content,
              format_polygon(line.polygon),
          )
      )
    for word in page.words:
      print(
          "...Word'{}' has a confidence of {}".format(
              word.content,word.confidence
          )
      )
    for selection_mark in page.selection_marks:
      print(
          "...Selection mark is '{}' within bounding box '{}'and has a confidentce of {}".format(
              selection_mark.state,
              format_polygon(selection_mark.polygon),
              selection_mark.confidence,
          )
      )
  for table_idx,table in enumerate(result.tables):
    print(
        "Table#{} has {} rows and {} columns".format(
            table_idx,table.row_count,table.column_count
        )
    )
    for region in table.bounding_regions:
      print(
          "Table #{} location on page:{} is {}".format(
           table_idx,
           region.page_number,
           format_polygon(region.polygon),
          )
      )

    for cell in table.cells:
      print(
        "...Cell[{}][{}] has content '{}'".format(
            cell.row_index,
            cell.column_index,
            cell.content,
            )
      )
      for region in cell.bounding_regions:
        print(
          "...content on page {} is within bounding box '{}'\n".format(
              region.page_number,
              format_polygon(region.polygon),
          )
        )
  print("-------------------")
if __name__=="__main__":
  analyze_general_documents()


Document contains handwritten content:
X,Yes
----key-value pair found in document----
key ':selected: QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ' found within 'Page #1:[0.9837,1.7426],[7.841,1.7426],[7.841,2.0625],[0.9837,2.0625]' bounding regions
value 'uer's clas' found within 'Page #1:[0.6572,1.7653],[0.7809,1.7653],[0.7809,1.8873],[0.6572,1.8873]' bounding regions 

key 'CT OF 1934
For the Quarterly P' found within 'Page #1:[0.9837,2.1341],[2.6264,2.1341],[2.6264,2.2773],[0.9837,2.2773]' bounding regions
value 'riod Ended Mar' found within 'Page #1:[2.6551,2.1341],[3.4573,2.1341],[3.4573,2.2773],[2.6551,2.2773]' bounding regions 

key '1, 2020
OR :unselected: TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURIT' found within 'Page #1:[0.9837,2.6688],[7.8506,2.6688],[7.8506,2.9935],[0.9837,2.9935]' bounding regions
value 'es of common' found within 'Page #1:[0.6611,2.6871],[0.7835,2.6871],[0.7835,2.8103],[0.6611,2.8103]' bounding regions